In [1]:
import torch

In [2]:
x = torch.rand(3,5)

In [3]:
x

tensor([[0.6377, 0.5303, 0.8268, 0.9287, 0.1826],
        [0.6539, 0.0270, 0.4157, 0.3265, 0.1726],
        [0.6405, 0.0505, 0.0141, 0.6024, 0.7342]])

In [4]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib.pyplot as plt

In [5]:
# Download training data from open datasets.
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [8]:
for X, y in test_loader:
    print(f'Shape of X[N,C,H,W]: {X.shape}')
    print(f'Shape of y: {y.shape, y.type}')
    break

Shape of X[N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: (torch.Size([64]), <built-in method type of Tensor object at 0x7fb6acdfacc0>)


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch %100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [13]:
def test(dataloader, model, loos_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f'Test Error: \nAccuracy: {(100*correct): >0.1f} %, Avg.loss {test_loss: >8f}')

In [14]:
epochs = 5

for t in range(epochs):
    print(f'Epochs {t+1}\n ---------------')
    train(train_loader,model, loss_fn, optimizer)
    test(test_loader,model, loss_fn)
    print('Done')

Epochs 1
 ---------------
loss: 2.300229 [    0/60000]
loss: 2.282672 [ 6400/60000]
loss: 2.273731 [12800/60000]
loss: 2.263159 [19200/60000]
loss: 2.247338 [25600/60000]
loss: 2.225632 [32000/60000]
loss: 2.226858 [38400/60000]
loss: 2.197253 [44800/60000]
loss: 2.184485 [51200/60000]
loss: 2.158143 [57600/60000]
Test Error: 
Accuracy: 52.8 %, Avg.loss 2.157298
Done
Epochs 2
 ---------------
loss: 2.163103 [    0/60000]
loss: 2.156682 [ 6400/60000]
loss: 2.106363 [12800/60000]
loss: 2.120665 [19200/60000]
loss: 2.079228 [25600/60000]
loss: 2.018616 [32000/60000]
loss: 2.043540 [38400/60000]
loss: 1.968733 [44800/60000]
loss: 1.962046 [51200/60000]
loss: 1.899745 [57600/60000]
Test Error: 
Accuracy: 61.2 %, Avg.loss 1.905214
Done
Epochs 3
 ---------------
loss: 1.926411 [    0/60000]
loss: 1.907553 [ 6400/60000]
loss: 1.795595 [12800/60000]
loss: 1.836089 [19200/60000]
loss: 1.735938 [25600/60000]
loss: 1.678936 [32000/60000]
loss: 1.692285 [38400/60000]
loss: 1.593658 [44800/60000]
lo

In [15]:
# save model
torch.save(model.state_dict(), 'model.pth')


In [16]:
# load model
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [17]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
